# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room Part #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
%pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

Note: you may need to restart the kernel to use updated packages.


We'll also get the "star of the show" today, which is Ragas!

In [2]:
%pip install -qU ragas

Note: you may need to restart the kernel to use updated packages.


We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
%pip install -qU qdrant-client pymupdf pandas

Note: you may need to restart the kernel to use updated packages.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

# from getpass import getpass
# openai.api_key = getpass("Please provide your OpenAI Key: ")
# os.environ["OPENAI_API_KEY"] = openai.api_key

True

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "../../../DataRepository/tswift_fued.pdf",
)

documents = loader.load()
len(documents)

22

In [6]:
documents[0].metadata

{'source': '../../../DataRepository/tswift_fued.pdf',
 'file_path': '../../../DataRepository/tswift_fued.pdf',
 'page': 0,
 'total_pages': 22,
 'format': 'PDF 1.4',
 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud",
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
 'producer': 'Skia/PDF m123',
 'creationDate': "D:20240423220523+00'00'",
 'modDate': "D:20240423220523+00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)
len(documents)

177

Let's confirm we've split our document.

In [8]:
len(documents)

177

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="Taylor Swift - Fued - ADA",
)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> - It uses HNSW algorithm for nearest neighbor selection
> - It uses the fastest Lloyd's k-means
> - It uses fastest small k-selection algorithm known
> - Optionally, it uses the fastest exact & approxicate NN search implementationn for high dim vectors (GPU required)

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("Who is Taylor Swift fueding with?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n19/22\nMUSIC" metadata={'source': '../../../DataRepository/tswift_fued.pdf', 'file_path': '../../../DataRepository/tswift_fued.pdf', 'page': 18, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240423220523+00'00'", 'modDate': "D:20240423220523+00'00'", 'trapped': '', '_id': '331aa2c8906f4fdfa000283eab9f7729', '_collection_name': 'Taylor Swift - Fued - ADA'}
page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n21/22\nMUSIC" metadata={'source': '../../.

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

> We first construct a dictionary of input user `question` and retrieved `context`. 
> We then passed both the prompt before handing the hydrated prompt to the `primary_qa_llm`.
> The output from the LLM is returned in a dictionary with key `response`, along with 
> another key `context` which stores the retrieved context.

Let's test it out!

In [18]:
question = "Who is Taylor Swift fueding with?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Kim Kardashian


In [19]:
question = "Why are they feuding?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n1/22\nPHOTO: KEVIN MAZUR/MTV1415/WIREIMAGE", metadata={'source': '../../../DataRepository/tswift_fued.pdf', 'file_path': '../../../DataRepository/tswift_fued.pdf', 'page': 0, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240423220523+00'00'", 'modDate': "D:20240423220523+00'00'", 'trapped': '', '_id': 'b3d781d789144cfb87f780979c845f84', '_collection_name': 'Taylor Swift - Fued - ADA'}), Document(page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room Part #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [20]:
loader = PyMuPDFLoader(
    "../../../DataRepository/tswift_fued.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

> This different-parameters approach is to avoid the synthetic data generator to use the same
> text chunk as the vector retriever and query system.

In [21]:
len(eval_documents)

56

> NOTE: This cell will take ~5-6min. to generate. If you run into any rate-limit issues - please use GPT-3.5-Turbo as your `critic_llm`. If you see any fields marked `nan` - this is product of rate-limiting issues, and you can safely ignore them for now.

In [23]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo") # <--- If you don't have GPT-4 access, or run into rate-limit, or `nan` issues.
# critic_llm = ChatOpenAI(model="gpt-4-turbo")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/112 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

max retries exceeded for SimpleEvolution(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=60, max_retries=15, max_wait=90, max_workers=16, exception_types=<class 'openai.RateLimitError'>)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x325f31cd0>, nodes=[Node(page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n1/22\nPHOTO: KEVIN MAZUR/MTV1415/WIREIMAGE\nA Timeline of Taylor Swift and Kim\nKardashian's Feud\nTake a look back at the drama between Taylor Swift and Kim Kardashian\nthrough the years\nBy  \n |  Updated on April 23, 2024 04:46PM EDT\nENTERTAINMENT\nMUSIC\nKelsie Gibson\nAdvertisement\nAd\ni\nThe Rewind: '13 Going 30'\nC LO S E\xa0\nSUBSCRIBE\nSKIP TO CONTENT", metadata={'source': '../../../DataRepository/tswift_fued.pdf', 'file_path': '../../../DataRepository/tswift_fued.pdf', 'page': 0, 'total_pages':

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How did the singer's experience affect her tru...,[Summer” singer said.\n“You have a fully manuf...,The singer's experience caused her to push awa...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
1,What was the significance of the secretly reco...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",The significance of the secretly recorded phon...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
2,What is the timeline of Taylor Swift and Kim K...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
3,How did Taylor Swift react to Kanye West infor...,"[""What I give a f--- is you as a person, and a...",Taylor Swift thanked Kanye West for informing ...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
4,What is the timeline of Taylor Swift and Kim K...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",Take a look back at the drama between Taylor S...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
5,Who are the inspirations for Taylor Swift's 'S...,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
6,How did the public's reaction to the leaked ph...,"[song. \nDecember 7\n, 2023: Taylor Swift says...",The public's reaction to the leaked phone call...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
7,What was the outcome of West's controversial r...,[discussed the longstanding feud between West ...,nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
8,What is the focus of the advertisement for '13...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
9,How did Taylor Swift reference the drama with ...,[In the music video for the song’s lead single...,In the music video for the song’s lead single ...,simple,[{'source': '../../../DataRepository/tswift_fu...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> This is the fractional distribution of synthetic Q-A dataset generation:
> - simple questions: 50%
> - questions requiring reasoning: 25%
> - questions requiring multiple contexts (chunks) to answer: 25%

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [27]:
from pprint import pprint
pprint(vars(testset.test_data[0]), indent=4)

{   'contexts': [   'Summer” singer said.\n'
                    '“You have a fully manufactured frame job, in an illegally '
                    'recorded phone\n'
                    'call, which Kim Kardashian edited and then put out to say '
                    'to everyone that\n'
                    'I was a liar,” she recalled.\xa0\n'
                    '“That took me down psychologically to a place I’ve never '
                    'been before,” the\n'
                    'Grammy winner continued. “I moved to a foreign country. I '
                    'didn’t leave a\n'
                    'rental house for a year. I was afraid to get on phone '
                    'calls. I pushed away\n'
                    'most people in my life because I didn’t trust anyone '
                    'anymore. I went down\n'
                    'really, really hard.”\n'
                    'Advertisement\n'
                    'Ad\n'
                    'i\n'
                    "The Rewind: '

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [30]:
test_df = testset.to_pandas()

In [31]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How did the singer's experience affect her tru...,[Summer” singer said.\n“You have a fully manuf...,The singer's experience caused her to push awa...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
1,What was the significance of the secretly reco...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",The significance of the secretly recorded phon...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
2,What is the timeline of Taylor Swift and Kim K...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
3,How did Taylor Swift react to Kanye West infor...,"[""What I give a f--- is you as a person, and a...",Taylor Swift thanked Kanye West for informing ...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
4,What is the timeline of Taylor Swift and Kim K...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",Take a look back at the drama between Taylor S...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
5,Who are the inspirations for Taylor Swift's 'S...,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
6,How did the public's reaction to the leaked ph...,"[song. \nDecember 7\n, 2023: Taylor Swift says...",The public's reaction to the leaked phone call...,simple,[{'source': '../../../DataRepository/tswift_fu...,True
7,What was the outcome of West's controversial r...,[discussed the longstanding feud between West ...,nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
8,What is the focus of the advertisement for '13...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,simple,[{'source': '../../../DataRepository/tswift_fu...,True
9,How did Taylor Swift reference the drama with ...,[In the music video for the song’s lead single...,In the music video for the song’s lead single ...,simple,[{'source': '../../../DataRepository/tswift_fu...,True


In [32]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [33]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [34]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [35]:
response_dataset[0]

{'question': "How did the singer's experience affect her trust in others?",
 'answer': "The singer's experience made her afraid to get on phone calls and pushed away most people in her life because she didn't trust anyone anymore.",
 'contexts': ['rental house for a year. I was afraid to get on phone calls. I pushed away\nmost people in my life because I didn’t trust anyone anymore. I went down\nreally, really hard.”\nAdvertisement\nAd\ni',
  'Swift said. "I mean, I don\'t think anybody would listen to that and be like,\n\'Oh, that\'s a real diss.\' You gotta tell the story that way that it happened to\nyou and the way you experienced it."',
  'like “a career death” and “took [her] down psychologically to a place\n[she’d] never been before.”\n“Make no mistake — my career was taken away from me,” the “Cruel\nSummer” singer said.',
  '11/22\nPHOTO: KEVIN WINTER/GETTY\nIn a candid cover story interview with Elle, Swift alluded to her rocky\nrelationship with West and Kardashian and spoke 

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [36]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [37]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


In [38]:
results

{'faithfulness': 0.9722, 'answer_relevancy': 0.5359, 'context_recall': 0.5789, 'context_precision': 0.5746, 'answer_correctness': 0.4964}

In [39]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How did the singer's experience affect her tru...,The singer's experience made her afraid to get...,[rental house for a year. I was afraid to get ...,The singer's experience caused her to push awa...,1.000000,0.882615,1.0,0.833333,0.991829
1,What was the significance of the secretly reco...,The secretly recorded phone call between Kanye...,"[never asked to be a part of, since 2009,"" Swi...",The significance of the secretly recorded phon...,1.000000,0.973647,1.0,1.000000,0.612732
2,What is the timeline of Taylor Swift and Kim K...,I don't know,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,NaN,0.000000,0.0,0.000000,0.197624
3,How did Taylor Swift react to Kanye West infor...,"Taylor Swift responded with a laugh and said, ...","[“Famous” lyric all along \n""She totally appro...",Taylor Swift thanked Kanye West for informing ...,1.000000,0.934969,0.0,1.000000,0.885193
4,What is the timeline of Taylor Swift and Kim K...,I don't know,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",Take a look back at the drama between Taylor S...,NaN,0.000000,0.0,0.000000,0.185126
5,Who are the inspirations for Taylor Swift's 'S...,I don't know.,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,NaN,0.000000,0.0,0.000000,0.198199
6,How did the public's reaction to the leaked ph...,The public's reaction to the leaked phone call...,[infamously leaked an edited phone call betwee...,The public's reaction to the leaked phone call...,1.000000,0.988759,1.0,0.805556,0.740679
7,What was the outcome of West's controversial r...,I don't know.,[discussed the longstanding feud between West ...,nan,NaN,0.000000,0.0,0.000000,0.198202
8,What is the focus of the advertisement for '13...,I don't know.,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,NaN,0.000000,0.0,0.000000,0.198202
9,How did Taylor Swift reference the drama with ...,Taylor Swift referenced the drama with Kanye W...,"[then Kim posts it on the Internet.""\nAugust 2...",In the music video for the song’s lead single ...,1.000000,0.924883,1.0,1.000000,0.522475


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [40]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [42]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [44]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift feuding with?"})

In [45]:
print(response["answer"])

Taylor Swift is feuding with Kim Kardashian.


In [46]:
response = retrieval_chain.invoke({"input": "Why are they feuding?"})

In [47]:
print(response["answer"])

The feud between Taylor Swift and Kim Kardashian started over leaked phone calls and escalated over time, with both parties publicly addressing and reacting to the situation. However, as of the latest update, Kim Kardashian mentioned that she was "over it," indicating that the feud may have come to an end.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [48]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [49]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [50]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


In [51]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How did the singer's experience affect her tru...,The singer's experience affected her trust in ...,[rental house for a year. I was afraid to get ...,The singer's experience caused her to push awa...,1.000000,0.999999,1.00,0.750000,0.996318
1,What was the significance of the secretly reco...,The secretly recorded phone call between Kanye...,"[March 20, 2020: The full phone call\nbetween ...",The significance of the secretly recorded phon...,1.000000,0.988475,1.00,1.000000,0.905779
2,What is the timeline of Taylor Swift and Kim K...,"I'm sorry, but I cannot provide the timeline o...","[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,NaN,0.000000,0.00,0.000000,0.178742
3,How did Taylor Swift react to Kanye West infor...,Taylor Swift reacted with a laugh when Kanye W...,"[song, the pop star appeared to be nervous as ...",Taylor Swift thanked Kanye West for informing ...,1.000000,0.998194,0.00,1.000000,0.726000
4,What is the timeline of Taylor Swift and Kim K...,The timeline of Taylor Swift and Kim Kardashia...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",Take a look back at the drama between Taylor S...,0.750000,1.000000,0.00,0.000000,0.223768
5,Who are the inspirations for Taylor Swift's 'S...,The context does not provide specific informat...,[Who Are Taylor Swift's 'Speak Now' Songs Abou...,nan,NaN,0.000000,0.00,0.000000,0.180247
6,How did the public's reaction to the leaked ph...,The leaked phone call between Taylor Swift and...,[video showed nothing new. We all knew that. I...,The public's reaction to the leaked phone call...,1.000000,0.978026,1.00,0.679167,0.834456
7,What was the outcome of West's controversial r...,The outcome of West's controversial rant on Sa...,[discussed the longstanding feud between West ...,nan,NaN,0.000000,0.00,0.000000,0.185769
8,What is the focus of the advertisement for '13...,The focus of the advertisement is on 'The Rewi...,[Advertisement\nAd\ni\nThe Rewind: '13 Going 3...,nan,1.000000,0.901252,0.00,0.000000,0.184704
9,How did Taylor Swift reference the drama with ...,Taylor Swift referenced the drama with Kanye W...,"[then Kim posts it on the Internet.""\nAugust 2...",In the music video for the song’s lead single ...,1.000000,0.976861,0.75,1.000000,0.652061


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [52]:
results

{'faithfulness': 0.9722, 'answer_relevancy': 0.5359, 'context_recall': 0.5789, 'context_precision': 0.5746, 'answer_correctness': 0.4964}

And see how our advanced retrieval modified our chain!

In [53]:
advanced_retrieval_results

{'faithfulness': 0.8698, 'answer_relevancy': 0.7957, 'context_recall': 0.6184, 'context_precision': 0.4803, 'answer_correctness': 0.5833}

In [54]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.972222,0.869792,-0.102431
1,answer_relevancy,0.535853,0.795727,0.259874
2,context_recall,0.578947,0.618421,0.039474
3,context_precision,0.574561,0.480253,-0.094309
4,answer_correctness,0.496401,0.583273,0.086873


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [56]:
# let's try out the new OpenAI `text-embedding-3-small` vs the ada embedding
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [57]:
# construct a vector_store for the input documents using QDrant and the new text-embedding-3-small embedding
vector_store = qdrant = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="Taylor Swift - Fued - MQR",
)

In [58]:
# get a chunk retriever from the Qdrant vector store
new_retriever = vector_store.as_retriever()

In [59]:
# create an advanced Multi-Query retriever based on the simple Qdrant vector retriever
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [60]:
# with the advanced retriever, construct a new chain with the new embedding
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [61]:
# create answers to the synthetic data set using the new retriever with the new embedding
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [62]:
# construct the dataset format needed for RAGAS
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [63]:
# invoke RAGAS to evaluate the results with the new embedding retriever
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.


In [64]:
# let's look at the raw result in text
new_advanced_retrieval_results

{'faithfulness': 0.8627, 'answer_relevancy': 0.8408, 'context_recall': 0.5526, 'context_precision': 0.5303, 'answer_correctness': 0.5488}

In [65]:
# create a merge result set for viewing using Pandas
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.972222,0.869792,0.862745,-0.007047,-0.109477
1,answer_relevancy,0.535853,0.795727,0.840804,0.045076,0.304951
2,context_recall,0.578947,0.618421,0.552632,-0.065789,-0.026316
3,context_precision,0.574561,0.480253,0.530315,0.050063,-0.044246
4,answer_correctness,0.496401,0.583273,0.548762,-0.034511,0.052361


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

> Based on the simple synthetic RAGAS evaluation, the improvement with the new embedding on the
> Taylor Swift data set seems very small (about 4.5% for `answer_relevance` and 5% for `context precision`).
> At the same time, there has been a decrease of 6.5% in `context_recall`, a decrease of 0.7% in faithfulness,
> and a decrease of 3.4% in answer_correctness. 
> On balance, we conclude that the improvement is not *significantly better than* `ada`

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [75]:
test_df.columns

Index(['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata',
       'episode_done'],
      dtype='object')

In [92]:
multi_context_results = evaluate(response_dataset.select(indices=range(10, 17)), metrics)

Evaluating:   0%|          | 0/35 [00:00<?, ?it/s]

No statements were generated from the answer.


In [93]:
multi_context_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.7827, 'context_recall': 0.7857, 'context_precision': 0.7302, 'answer_correctness': 0.5249}

In [96]:
response_dataset_advanced_retrieval.shape

(19, 4)

In [97]:
advanced_multi_context_results = evaluate(response_dataset_advanced_retrieval.select(indices=range(10, 17)), metrics)

Evaluating:   0%|          | 0/35 [00:00<?, ?it/s]

In [98]:
advanced_multi_context_results

{'faithfulness': 0.7857, 'answer_relevancy': 0.9212, 'context_recall': 1.0000, 'context_precision': 0.6422, 'answer_correctness': 0.6544}

## Observation on `multi-context` questions

The advanced multiquery retriever clearly achieved much better `answer_relevancy` and `context_recall`.  However, there is a significant drop in `faithfulness` (in other words, there seems to be a significant rise of hallucination).  The multi-query retriever seems to expand the original query enough to expand the scope of the answer responses -- which seem to lead to an increase in made up answers to the original user query. This result deserves thoughtful further investigation on whether the increased performance is worth the risk of hallucination.